In [4]:
def get_all_not_used_triplets(string_token):
    wikidata_all_triplets = open("/data/fichtel/projektarbeit/gold_dataset.nt", "r")
    dictio_prop_template = json.load(open("/data/fichtel/projektarbeit/templates.json", "r"))

    #collect the triplets of wikidata in a dict
    dictio_prop_triple = {}
    for line in wikidata_all_triplets:
        triple = ((line.replace("\n", "")).replace(".", "")).split(" ")
        prop = str(triple[1]).split('/')[-1].replace('>', "")
        if prop in dictio_prop_template:
            subj = str(triple[0]).split('/')[-1].replace('>', "")
            obj = str(triple[2]).split('/')[-1].replace('>', "")
            if prop not in dictio_prop_triple:
                dictio_prop_triple[prop] = set()
            dictio_prop_triple[prop].add((subj, obj))
    #get the used triplets of the queries
    dictio_prop_query_answer = json.load(open("/data/fichtel/projektarbeit/queries_{}_all.json".format(string_token), "r"))       
    dictio_prop_used_triple = {}
    for prop in dictio_prop_query_answer:
        dictio_prop_used_triple[prop] = set()
        #get query
        for query in dictio_prop_query_answer[prop]:
            for answer in dictio_prop_query_answer[prop][query]:
                triple = query.replace("?", answer).split("_")
                subj = triple[0]
                obj = triple[2]
                subj_label = dictio_id_label[subj][0]
                obj_label = dictio_id_label[obj][0]
                dictio_prop_used_triple[prop].add((subj_label, obj_label))
    #get the not used triple to use them to find the sentences for the templates
    dictio_prop_not_used_triple = {}
    for prop in dictio_prop_triple:
        not_used_triple = dictio_prop_triple[prop] - dictio_prop_used_triple[prop]
        dictio_prop_not_used_triple[prop] = not_used_triple
    
    return dictio_prop_not_used_triple

def get_ents_in_sent(
        sent_b,
        ents,
        mandatory_uri_substr="wikidata.org/entity/"):
    # filter entities which are in boundaries
    # and whose uris contain the mandatory uri substring
    # (default: to filter only entities, not relations)
    ents_copy = copy.deepcopy(ents)
    sent_ents = list(filter(
        lambda ent: (
            ent["boundaries"][0] >= sent_b[0] and
            ent["boundaries"][0] < sent_b[1] and
            mandatory_uri_substr in ent["uri"]
            ),
        ents_copy
        ))

    # align boundaries from abstract level to sentence level
    for i in range(0, len(sent_ents)):
        sent_ents[i]["boundaries"][0] -= sent_b[0]
        sent_ents[i]["boundaries"][1] -= sent_b[0]
            
    return sent_ents

In [5]:
import json
import sys
import os
import copy

string_token = "onetoken"
lm_name = "bert-base-cased"
dictio_id_label = json.load(open("/data/fichtel/projektarbeit/entity2label_onlyrdflabel.json", "r"))
dictio_prop_all_triple = get_all_not_used_triplets(string_token)

In [10]:
def index_sentences(dictio_id_label, dictio_prop_triple):
    maxLength = 10
    index = {}
    for i, file in enumerate(os.listdir("/data/fichtel/projektarbeit/trex/")):
        print("index sentences in file {}".format(i))
        path = os.path.join("/data/fichtel/projektarbeit/trex/", file)
        trex_dataset = json.load(open(path, "r"))
        # go through every abstract associated to an entity
        for abstract in trex_dataset:
            # get sentences from "sentences_boundaries"
            for sent_b in abstract["sentences_boundaries"]:
                sent = abstract["text"][sent_b[0]:sent_b[1]]
                if len(sent.split()) <= maxLength:
                    # get entities in this boundary
                    sent_ents = get_ents_in_sent(sent_b, abstract["entities"])
                    # index sentences containing entity pairs (e1,e2) from input relation r
                    if len(sent_ents) == 2:
                        for e1 in sent_ents:
                            for e2 in sent_ents:
                                if e1 != e2:
                                    try:
                                        e1['uri'] = e1['uri'].replace("http://www.wikidata.org/entity/", "")
                                        e2['uri'] = e2['uri'].replace("http://www.wikidata.org/entity/", "")
                                        #check for sentence whether it belongs to one of the input relations in props
                                        for prop in dictio_prop_triple:
                                            entityPairs = dictio_prop_triple[prop]
                                            if (e1['uri'],e2['uri']) in entityPairs:
                                                    #check if entities are overlapping (e.g. "This novel won the John Newbery Medal in 1996.", e1=John Newbery, e2=Newbery Medal)
                                                overlapping = False
                                                if e1['boundaries'][0] < e2['boundaries'][0] and e2['boundaries'][0] <= e1['boundaries'][1]:
                                                    overlapping = True
                                                elif e1['boundaries'][0] > e2['boundaries'][0] and e1['boundaries'][0] <= e2['boundaries'][1]:
                                                    overlapping = True
                                                #check if label extracted with boundaries is equal to surfaceform
                                                e1_too_short_surfaceform = False
                                                if e1['boundaries'][1] - e1['boundaries'][0] != len(dictio_id_label[e1['uri']][0]):
                                                    e1_label_token = dictio_id_label[e1['uri']][0].split(" ")
                                                    for token in e1_label_token:
                                                        if token in e1["surfaceform"].split(" "):
                                                            e1_too_short_surfaceform = True
                                                            break
                                                e2_too_short_surfaceform = False
                                                if e2['boundaries'][1] - e2['boundaries'][0] != len(dictio_id_label[e2['uri']][0]):
                                                    e2_label_token = dictio_id_label[e2['uri']][0].split(" ")
                                                    for token in e2_label_token:
                                                        if token in e2["surfaceform"].split(" "):
                                                            e2_too_short_surfaceform = True
                                                            break
                                                if e1_too_short_surfaceform == False and e2_too_short_surfaceform == False and overlapping == False:
                                                    entry = {}
                                                    #replace the current surfaceform with the label of wikidata, which are used
                                                    if e1['boundaries'][0] < e2['boundaries'][0]:
                                                        sentence = sent[:e1['boundaries'][0]] + dictio_id_label[e1['uri']][0] + sent[e1['boundaries'][1]:e2['boundaries'][0]] + dictio_id_label[e2['uri']][0] + sent[e2['boundaries'][1]:]
                                                    elif e1['boundaries'][0] > e2['boundaries'][0]:
                                                        sentence = sent[:e2['boundaries'][0]] + dictio_id_label[e2['uri']][0] + sent[e2['boundaries'][1]:e1['boundaries'][0]] + dictio_id_label[e1['uri']][0] + sent[e1['boundaries'][1]:]
                                                    else:
                                                        sentence = -1
                                                        print("subject==object --> sentence: {}, surfaceform: {}".format(sent ,e1["surfaceform"]))
                                                    if sentence != -1:
                                                        entry['sentence'] = sentence
                                                        entry['entities'] = (e1['uri'],e2['uri'])

                                                        #adjust boundaries
                                                        e1_copy = copy.deepcopy(e1)
                                                        e2_copy = copy.deepcopy(e2)
                                                        if e1_copy['boundaries'][0] < e2_copy['boundaries'][0]:
                                                            e1_boundary_diff = len(dictio_id_label[e1_copy['uri']][0]) - (e1_copy['boundaries'][1] - e1_copy['boundaries'][0])
                                                            entry['e1'] = e1_copy['boundaries'][:]
                                                            entry['e1'][1] = entry['e1'][0] + len(dictio_id_label[e1_copy['uri']][0])
                                                            entry['e2'] = e2_copy['boundaries']
                                                            entry['e2'][0] = entry['e2'][0] + e1_boundary_diff
                                                            entry['e2'][1] = entry['e2'][0] + len(dictio_id_label[e2_copy['uri']][0])
                                                        elif e1_copy['boundaries'][0] > e2_copy['boundaries'][0]:
                                                            e2_boundary_diff = len(dictio_id_label[e2_copy['uri']][0]) - (e2_copy['boundaries'][1] - e2_copy['boundaries'][0])
                                                            entry['e2'] = e2_copy['boundaries'][:]
                                                            entry['e2'][1] = entry['e2'][0] + len(dictio_id_label[e2_copy['uri']][0])
                                                            entry['e1'] = e1_copy['boundaries']
                                                            entry['e1'][0] = entry['e1'][0] + e2_boundary_diff
                                                            entry['e1'][1] = entry['e1'][0] + len(dictio_id_label[e1_copy['uri']][0])
                                                        if prop in index:
                                                            index[prop].append(entry)
                                                        else:
                                                            index[prop] = [entry]
                                                        if entry['e1'][0] < entry['e2'][0]:
                                                            subject_object_template = sentence[:entry['e1'][0]] + "[S]" + sentence[entry['e1'][1]:entry['e2'][0]] + "[O]" + sentence[entry['e2'][1]:]
                                                        else:
                                                            subject_object_template = sentence[:entry['e2'][0]] + "[O]" + sentence[entry['e2'][1]:entry['e1'][0]] + "[S]" + sentence[entry['e1'][1]:]
                    
                                                        print(subject_object_template)
                                    except KeyError:
                                        print("WARNING: KeyError")
                                        continue
    return index

index = index_sentences(dictio_id_label, dictio_prop_all_triple)

index sentences in file 0
[S] was Released by [O] on May 17, 2005.
These postcode districts cover north-west [S] and much of [O].
These postcode districts cover north-west [O] and much of [S].
[S] was born in [O].
[O] is currently the guitarist of German metal band [S].
[S] is currently the guitarist of German metal band [O].
[S] played for [O] from 1941 to 1950.
In the early 1990s, [S] and [O] also separated.
In the early 1990s, [S] and [O] also separated.
In the early 1990s, [O] and [S] also separated.
In the early 1990s, [O] and [S] also separated.
[O] band JUDICIAL [S] from 1980-1984.
Morgan signed with [O] [S] team Södertälje SK in 2008.
In 1864 he explored northern [S] and southeastern [O].
In 1864 he explored northern [O] and southeastern [S].
[S] joined [O] in 1992.
[S] Stadium, [O].
[S] Stadium, [O].
[S] Stadium, [O].
[S] are an Australian [O] band which formed in 1980.
[S] had left to join [O].
[O] had left to join [S].
[S] served as [O] of Trois-Rivières in 1845 and 1853.
Ja

[S] died in [O] in 1877.
[S] died in [O] in 1877.
[S] is now in the [O].
[S] died in [O], aged 60.
[S] died in [O], aged 60.
[S] was made part of [O] on 1 January 1948.
[S] is a [O] country.
[O] is a [S] country.
[S] currently plays as a [O].
[S] State was a princely state in [O].
[S] is the son of [O] Max D. Barnes.
[S] (矢田亜希子 Yada Akiko) is a Japanese [O].
[S] died on January 29, 1935 in [O].
[S] has created some of [O]'s hit songs.
[S] was born in [O].
[S] went to [O] and graduated in 1882.
from the state of [S], [O].
from the state of [S], [O].
[S] represented [O] state.
[S] were subsequently signed to [O]
Its offices are in Square de Meeus, [S], [O].
Its offices are in Square de Meeus, [S], [O].
[S] died on 1 October 1962 at [O].
[S] is managed by the [O].
[S] is a British television and theatre [O].
[S] played for the 1898 [O].
[S] is also a notable attendee at [O].
The [O] was [S].
[S] (died 1840) was an [O] antiquary.
[S] was born in [O].
[S] is situated in the Southern Distric

[S] is owned by [O]
Route 395 just east of [S] in [O].
Route 395 just east of [S] in [O].
Route 395 just east of [O] in [S].
from the [S] in [O].
from the [S] in [O].
[S] died on 5 February 1946 in [O].
Harris is buried in [O]'s [S].
[S] was a finalist of the 2011 [O].
[S] was awarded a [O] in 2003.
[S] ([O] האושפיזין) (lit.
[S] died in [O] in 1864.
Since 2005, the company has belonged to [O]'s [S] group.
[S] died at [O] in 1984.
[S] died at [O] in 1984.
[O] is where the [S] took place in 1752.
His twin brother [S] is also a [O].
Gerasimov is buried in the [S] of [O].
Perak Tengah is an administrative district in [S], [O].
Perak Tengah is an administrative district in [S], [O].
[S] is the most populated state in [O].
[S] is the most populated state in [O].
[S] returned to [O] in the 1980s.
[S] is also [O]'s oldest surviving business.
[S] ([O]: موتی محل) Pearl Palace, may refer to:
Engines were also by [S], another [O] brand.
[S] is a former Central [O] Debating Champion.
That is, other

[S] died in [O] on April 12, 2008, aged 86.
[S] was born in [O].
[S] is also an occasional [O].
[S] was born in [O] in 1813.
[S] died in [O], aged 76.
[S] is a recipient of the [O].
[S] was released on June 6, 1995 through [O].
[S] was appointed coach of [O] in 2007.
1) in [S], [O].
1) in [S], [O].
1) in [O], [S].
[S] previously played for [O].
The [O] was [S].
[S] ([O]: මතුගම) is a town.
[S] was born in [O].
at [S], [O].
[S] is one of the largest industrial areas of [O].
Kanda-Hirakawachō (神田平河町) is a district of [S], [O].
[S] played for [O].
[S] was an [O] clergyman.
[S] was born in 1954 in [O].
[S] died in [O], aged 62.
[S] was released in 2002 by [O].
[S] was released 1995 by [O].
[S] was released 1989 by [O].
[S] died in [O] in 1945, aged 63.
[S] studied at [O].
[S] is a bus operator based in [O].
[S] was born in [O].
[S] last played for [O].
436, [S], [O].
436, [S], [O].
Bruni starred in the [O] [S] as Dr.
[S] was born in [O].
[S] was born in [O].
[S] was married to the dancer an

[S] wrote one [O], Hangjegyek lázadása (1931).
[O] is a local newspaper published in [S].
His [O] professors were Algernon Ashton and [S].
Deneen's great grandson is [O] [S].
[O] is the capital of the [S].
[S] is the capital of the [O].
[S] is the capital of the [O].
[S] died, aged 71, on 21 June 1970 in [O].
[O] currently produces [S] for running 3rd party firmwares.
[S] was released in 1988 by [O].
[S] is a political worker and [O].
The character was played by the [O] [S].
Brian Faloon is a musician born in [S], [O].
Brian Faloon is a musician born in [O], [S].
[O] is the head town of [S].
[S] is the head town of [O].
[S] is the head town of [O].
[S] is also major tourist attraction in [O].
[S] has recorded more than 70 of his [O] compositions.
state of [O], located in [S].
state of [O], located in [S].
state of [O] located in [S].
Ormberget is the highest mountain in Notgården, [S], [O].
state of [O] located in [S].
[S] is one of the longest rivers in [O].
state of [O], located in [

...

[S] took an early interest in natural history, particularly [O].
[S] became Professor of [O] in 1952.
The Sportwagonadvertising campaign was made with [O] [S].
Evrard lives and works in [S], [O].
[S] was born in [O].
state, as well as [O] through [S].
His nephew [S] is an [O] and producer.
index sentences in file 49
[S] (1897–1975) was a British [O].
[S] plays for [O] .
[S] started out on the streets of [O].
[S] was born in [O].
[S] belongs to [O] region .
[S], known in [O] as Shinryaku!
[S] additionally published many works on the resistance in [O].
The audio book is narrated by [O] [S].
Hart was born in [S], [O] in 1888.
The event took place in [S], [O] in November 1994.
The event took place in [S], [O] in November 1994.
The event took place in [S], [O] in November 1994.
The event took place in [O], [S] in November 1994.
[S] was released on 22 April 2011 on [O].
[S] died in October 1967 in [O].
Port [O] or Port [S] may refer to:
[S] (1901–1966) was a British film and radio [O].
[S] w

[S] died in [O] in his early to mid-fifties.
[S] is 15 km away from [O].
[S] was the brother of [O] Charles K. Gerrard.
[S] was released in [O] on 28 December 2007.
Flemming is the younger brother of Canadian [O] [S].
[S] ruled from the ancient walled city of [O].
[S] ruled from the ancient walled city of [O].
[S] died in [O].
María José Torrenegra Ariza ([S], [O], 27 October 1984).
Al Shamsi was [S]'s former ambassador to [O].
Al Shamsi was [O]'s former ambassador to [S].
Calder is a residential neighbourhood in northwest [S], [O].
Oeil de Perdrix is a rosé [O] produced in [S].
Breeza is a locality in [S], [O].
Breeza is a locality in [S], [O].
[S] is owned by the Village of [O].
[S] is owned by the [O] Airport Authority.
[S] is a band from [O].
[S] died in [O] in 1980.
[S] died in [O] in 1992.
[S] died in [O] in 1992.
Norelius was born in [S], [O] in 1909.
Norelius was born in [S], [O] in 1909.
Norelius was born in [O], [S] in 1909.
[S] is part of the [O] Area School District.
Her [O

[S] is a neighbourhood in [O].
The agency is located in [S], capital of [O].
The agency is located in [S], capital of [O].
The agency is located in [S], capital of [O].
The agency is located in [O], capital of [S].
Germanía survives today in the [S] [O].
[S] is considered one of [O]'s highest assets in music.
[S] was born in [O].
Common phrases for the [S] area of northern [O] include:
[S] was also Bishop of [O] from 1106 to 1112.
In 1935, [S] and [O] corporations began mining operations there.
In 1935, [O] and [S] corporations began mining operations there.
The [S] ended in September 1944, with [O]'s capitulation.
The [S] is the [O] of the U.S.
Kitaazumi (北安曇郡 Kita-Azumi-gun) is a district in [S], [O].
Kitaazumi (北安曇郡 Kita-Azumi-gun) is a district in [S], [O].
The town contains [S], the largest prison in [O].
[S] dramatically contributes to society in Obuse and [O].
[S] was released in 1985 on [O].
[S] won the 1990 [O].
[S] (born 1967) is a British [O] and producer.
[S] was released o

[S] also played for [O].
[S] died at [O] in 1702.
[S] died at [O] in 1702.
[S] was born in [O].
The Monumental Cemetery of Bonaria is located in [S], [O].
The Monumental Cemetery of Bonaria is located in [O], [S].
Its headquarters are in [S] in [O].
Kumkuyu is a town in [S], [O].
[S] died at [O] in 1704.
[S], [O] soldier and noble, died 1737.
The movie was dubbed into [O] as Super Hero [S].
Then she freelanced in [S] and [O].
Then she freelanced in [O] and [S].
[S] premiered on [O] on January 5, 2009.
[S] is also described as a [O] album.
The [S] or [O] is 743412-13.
Ayvagediği is a town in [S], [O].
His regular [O] is [S].
[S] also taught the [O] privately.
[S] died in [O] in 1876 in his late 30s.
[S] was born in [O].
[S] was [O]'s one of earliest experimental feature films.
The song is featured in the [O], [S].
Box 16650, [S], [O]
Box 16650, [S], [O]
Box 16650, [O], [S]
[S] was awarded the [O] in 1994.
[S] was awarded [O] for her achievements.
[S] currently resides in his hometown [O

KeyboardInterrupt: 